In [ ]:
%run lib/scrape_govuk.ipynb
metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')

import pyexcel
from io import BytesIO

ods_files = [f for f in metadata['files'] if f['type'] == 'ODS']

out = Path('out')
out.mkdir(exist_ok=True)

def load_sheet(ods_file_info, sheet_name):
    ods_file = BytesIO(session.get(ods_file_info['url']).content)
    return pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name=sheet_name)

for ods_file_info in ods_files:
    if ods_file_info['title'] == 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1':
        data = load_sheet(ods_file_info, 'vi_05')
        %run "Entry clearance visas granted by country of nationality(Vi_05).ipynb"
        entry_1 = tidy.copy()
        data = load_sheet(ods_file_info, 'vi_01_q')
        %run "Entry clearance visa applications and resolution by category ( vi_01_q).ipynb"
        entry_2 = tidy.copy()
        entry_1.rename(columns={'Year': 'Period'}, inplace=True)
        entry_1.replace(
            {'Period': r'$([0-9]{4})^'}, {'Period': r'year/\1'}, regex=True, inplace=True
        )
        for col in ['Type', 'Broad category', 'Applicant type', 'Category', 'Resolution']:
            entry_1[col] = 'All'
        # Todo: "World" or "Not UK"?
        entry_2['Country of nationality'] = 'World'
        pd.concat([entry_1, entry_2], sort=True).to_csv(out / 'entry_visas.csv', index=False)
        
        

,Geographical region,Country of nationality,Year,Value


['America North',
 'Cocos (Keeling) Islands',
 'American Samoa',
 'Korea (North)',
 'Burma',
 'Other and unknown',
 'Virgin Islands (British)',
 'Palau',
 'Bermuda (British)',
 'Asia Central',
 'Cook Islands',
 'EU 2',
 'Christmas Island',
 'Asia South',
 'Guinea-Bissau',
 'Curacao',
 'Occupied Palestinian Territories',
 'Asia South East',
 'Europe Other',
 'Refugee',
 'Virgin Islands (US)',
 'America Central and South',
 'Kosovo',
 'Kiribati',
 'St. Kitts and Nevis',
 'Asia East',
 'Bosnia and Herzegovina',
 'Central African Republic',
 'Micronesia',
 'Ireland',
 'Serbia and Montenegro',
 'Cayman Islands (British)',
 'Africa Sub-Saharan',
 'EU 14',
 'St. Martin (French Part)',
 'Sudan (South)',
 'Surinam',
 'St. Maarten (Dutch Part)',
 'Pitcairn Islands (British)',
 'Spain',
 'Netherlands Antilles',
 'Cambodia',
 'St. Lucia',
 'Macau',
 'Bonaire, Sint Eustatius and Saba',
 'EU Other',
 'Anguilla (British)',
 'Korea (South)',
 'Falkland Islands (British)',
 'Samoa',
 'EU 8',
 'Other',
